In [80]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNetCV

data_cleaned =  'https://raw.githubusercontent.com/mikdadkanbar/HousePricesML/main/data_cleaned.csv'

data_ready = 'https://raw.githubusercontent.com/mikdadkanbar/HousePricesML/main/data_ready.csv'


#data_ready is after creating the dummy variables 
df=pd.read_csv(data_ready)
df['YearRemodAdd'] = 2023 - df['YearRemodAdd']

df['SalePrice'] = df['SalePrice'] / 1000 
df.head() 


,Unnamed: 0,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,60,65.0,8450,7,5,20,20,196.0,706,...,0,0,0,0,1,0,0,0,1,0
1,1,20,80.0,9600,6,8,47,47,0.0,978,...,0,0,0,0,1,0,0,0,1,0
2,2,60,68.0,11250,7,5,22,21,162.0,486,...,0,0,0,0,1,0,0,0,1,0
3,3,70,60.0,9550,7,5,108,53,0.0,216,...,0,0,0,0,1,0,0,0,0,0
4,4,60,84.0,14260,8,5,23,23,350.0,655,...,0,0,0,0,1,0,0,0,1,0


In [82]:
#### original  code in R : 
# data_ready = 'https://raw.githubusercontent.com/mikdadkanbar/HousePricesML/main/data_ready.csv'

# df= read_csv(data_ready )
# df['YearRemodAdd'] = 2023-df['YearRemodAdd']
# df['SalePrice']= df['SalePrice'] / 1000

# set.seed(42)
# index= sample ( nrow(df), 0.2* nrow(df))
# test = df[index, ]
# train= df[-index,]

# x <- model.matrix(SalePrice ~ . -1, train)
# x_test <- model.matrix(SalePrice ~ . -1, test)
# y <- train$SalePrice
# set.seed(42)
# rl_mod <- cv.glmnet(x, y, relax = TRUE)

# rl_preds <- predict(rl_mod, s = "lambda.1se", newx = x_test)

# mse <-mean((test$SalePrice - rl_preds)^2)
# MAPE <- mean(abs( (test$SalePrice - rl_preds) /test$SalePrice)) * 100

# MAPE
# rl_coefs <- coef(rl_mod, s = "lambda.1se")
# #names (rl_coefs[rl_coefs[ , 1] != 0, ])
# rl_coefs[rl_coefs[ , 1] != 0, ]

# predict(rl_mod, s = "lambda.1se", newx  = as.matrix(df[1:10, !colnames(df) %in% "SalePrice"]))





In [81]:
#Because I did not find relaxed lasso in Python, I will make the model manually :



#only those are the variables I decided to include : 
newdf = df[['MSSubClass', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 
         'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', 'Fireplaces', 'GarageArea', 
         'RoomsAndArea', 'MSZoning_RM', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 
         'Neighborhood_StoneBr', 'ExterQual_TA', 'BsmtExposure_Gd', 'BsmtExposure_No', 
         'KitchenQual_TA', 'SaleType_New', 'SalePrice']]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(newdf.iloc[:, :-1], df.iloc[:, -1], test_size=0.2, random_state=42)



def predict_house_price(test_set):
    """
    Calculates SalePrice for  a dataframe ( that only contains the previously mentioned variables)

    Args:
        test_set (pd.DataFrame): Test set data containing features used in the model.

    Returns:
        SalePrice
    """

    # Coefficients from R results
    intercept = 28.3693757574
    coef_MSSubClass = -0.1181452467
    coef_LotArea = 0.0003794007
    coef_OverallQual = 15.4780609841
    coef_YearBuilt = -0.1228018977
    coef_YearRemodAdd = -0.2532649874
    coef_BsmtFinSF1 = 0.0189993635
    coef_TotalBsmtSF = 0.0171431519
    coef_1stFlrSF = -0.0042722873
    coef_Fireplaces = 7.6836473103
    coef_GarageArea = 0.0301962849
    coef_RoomsAndArea = 0.0038092336
    coef_MSZoning_RM = -3.9156803890
    coef_Neighborhood_NoRidge = 40.3861125289
    coef_Neighborhood_NridgHt = 31.8097617213
    coef_Neighborhood_StoneBr = 40.7972174908
    coef_ExterQual_TA = -5.6445368305
    coef_BsmtExposure_Gd = 14.4546062891
    coef_BsmtExposure_No = -4.9082239286
    coef_KitchenQual_TA = -5.5464880369
    coef_SaleType_New = 15.3338652349

    # Make predictions on the test set
    y_pred = (intercept +
              coef_MSSubClass * test_set['MSSubClass'] +
              coef_LotArea * test_set['LotArea'] +
              coef_OverallQual * test_set['OverallQual'] +
              coef_YearBuilt * test_set['YearBuilt'] +
              coef_YearRemodAdd * test_set['YearRemodAdd'] +
              coef_BsmtFinSF1 * test_set['BsmtFinSF1'] +
              coef_TotalBsmtSF * test_set['TotalBsmtSF'] +
              coef_1stFlrSF * test_set['1stFlrSF'] +
              coef_Fireplaces * test_set['Fireplaces'] +
              coef_GarageArea * test_set['GarageArea'] +
              coef_RoomsAndArea * test_set['RoomsAndArea'] +
              coef_MSZoning_RM * test_set['MSZoning_RM'] +
              coef_Neighborhood_NoRidge * test_set['Neighborhood_NoRidge'] +
              coef_Neighborhood_StoneBr * test_set['Neighborhood_StoneBr'] +
              coef_ExterQual_TA * test_set['ExterQual_TA'] +
    coef_BsmtExposure_Gd * test_set['BsmtExposure_Gd'] +
    coef_BsmtExposure_No * test_set['BsmtExposure_No'] +
    coef_KitchenQual_TA * test_set['KitchenQual_TA'] +
    coef_SaleType_New * test_set['SaleType_New'])
    
   
    return y_pred


predict_house_price(newdf.iloc[0:10,])

0    213.492353
1    193.347385
2    213.812761
3    180.641760
4    317.715701
5    142.920774
6    266.499434
7    219.851541
8    178.843227
9    111.850609
dtype: float64